# Markov Chain Simulation of M/M/k/k
Simulate an M/M/k/k queue based on the Markov-chain simulation principles to evaluate the blocking probability for a wide range of parameter values. Provide confidence intervals. Compare the simulation results with results obtained by the Erlang B Formula. (Slides P374)

In [162]:
import numpy as np
import random
import scipy.stats as stats
import pandas as pd

## Markov Chain Simulation
For the **M/M/k/k simulation**, Prof. Moshe provided a pseudo code on Slides P369-370. It is quite simple, and I am sure everyone can implement it easily.

In [163]:
def markov_chain_mmkk(k, lam, mu, MAXN):
    Q = 0 # queue size
    Na = 0 # number of arrivals so far
    Nb = 0 # number of blocked customers

    while Na < MAXN:
        if random.random() <= lam / (lam + Q * mu):
            Na += 1
            if Q == k:
                Nb += 1
            else:
                Q += 1
        else:
            Q -= 1
    Bp = Nb / Na

    return Bp

## Erlang B Formula
For the **Erlang B formula**, Slides P373 provides an easy way, **the Erlang B recursion**, to calculate.

A good way to implement the Erlang B recursion is by using **Dynamic Programming (DP)**.

There are **4 key steps** of dynamic programming we need to know before coding:

1. Find out the meaning of the DP array: in this case, DP[i] is the blocking probability of an M/M/i/i queue.

2. Find out the state-transition equation: in this case, the equation is $E_m\left(A\right)=\frac{AE_{m-1}\left(A\right)}{m+AE_{m-1}\left(A\right)}$.

3. Initialization of the DP array: in this case DP[0] = 1.

4. Find out the direction of the traversal: based on the state-transition equation, we should traverse from the beginning to the end.

**Tips:** Since we want to calculate the blocking probabilities of queues with server numbers from 0 to k, the size of the DP array should be **k + 1**.

In [164]:
def erlang_b_recursion(k, lam, mu):
    A = lam / mu
    E = np.zeros(k + 1)
    E[0] = 1
    for m in range(1, k + 1):
        E[m] = A * E[m - 1] / (m + A * E[m - 1])
    return E

## Comparison of the Blocking Probability

In [165]:
k = 10
lam = 15
mu = 5
total_arrivals = 50000
loop_time = 500

mean_block_probability = np.zeros(k + 1)
confidence_intervals = []
t_confidence_intervals = []
mean_block_probability[0] = 1
confidence_intervals.append([1, 1])
isInConfidenceIntervals = []

for avail_servers in range(1, k + 1):
    block_probability_markov_chain = np.zeros(loop_time)
    for i in range(loop_time):
        block_probability_markov_chain[i] = markov_chain_mmkk(avail_servers, lam, mu, total_arrivals)
    mean = np.mean(block_probability_markov_chain)
    mk_confidence_interval = stats.norm.interval(alpha = 0.95,
                                                 loc = mean,
                                                 scale = stats.sem(block_probability_markov_chain))
    mean_block_probability[avail_servers] = mean
    confidence_intervals.append(mk_confidence_interval)

block_probability_erlang_b_recursion = erlang_b_recursion(k, lam, mu)

for i in range(k + 1):
    if (block_probability_erlang_b_recursion[i] >= confidence_intervals[i][0]) and (block_probability_erlang_b_recursion[i] <= confidence_intervals[i][1]):
        isInConfidenceIntervals.append(True)
    else:
        isInConfidenceIntervals.append(False)

Parameters: $k=10$, $\lambda=15$, $\mu=5$, $MAXN_a=50000$, $LoopTime=500$. The table below shows the results.

The distances between the blocking probabilities are quite small. The results of Erlang B recursion all sit in the corresponding confidence intervals.

In [166]:
pd.DataFrame({
    "Available Servers": np.linspace(0, k, k + 1),
    "Erlang B Recursion Pb": block_probability_erlang_b_recursion,
    "Markov Chain Pb": mean_block_probability,
    "Markov Chain Norm Confidence Intervals": confidence_intervals,
    "IsInConfidenceIntervals": isInConfidenceIntervals,
})

,Available Servers,Erlang B Recursion Pb,Markov Chain Pb,Markov Chain Norm Confidence Intervals,IsInConfidenceIntervals
0,0.0,1.000000,1.000000,"[1, 1]",True
1,1.0,0.750000,0.750032,"(0.7498664184132776, 0.7501975015867225)",True
2,2.0,0.529412,0.529489,"(0.5292495283176231, 0.5297276716823767)",True
3,3.0,0.346154,0.346110,"(0.3458620705229983, 0.3463579294770018)",True
4,4.0,0.206107,0.206000,"(0.20578533796422327, 0.20621490203577675)",True
5,5.0,0.110054,0.110031,"(0.10984623950583265, 0.11021584049416737)",True
6,6.0,0.052157,0.052145,"(0.05201883891815743, 0.05227140108184256)",True
7,7.0,0.021864,0.021859,"(0.021771234322060292, 0.021945885677939706)",True
8,8.0,0.008132,0.008163,"(0.008108770529200534, 0.008216429470799468)",True
9,9.0,0.002703,0.002712,"(0.0026828406624241257, 0.0027415593375758746)",True
